In [1]:
import tweepy
import json
import pickle as pkl
import mysql.connector

In [26]:
# load 50 tweet ids
with open('top_tweetIds.pkl', 'rb') as f:
    tweet_ids = pkl.load(f)

In [27]:
tweet_ids = tweet_ids[150:]
tweet_ids = [str(id) for id in tweet_ids]
print(len(tweet_ids))

100


In [28]:
# Twitter API credentials
bearer_token = "AAAAAAAAAAAAAAAAAAAAAEeQyQEAAAAAz7RyNP5HHBjnPkdJap%2Fb%2Bna78Dc%3D1czDzE57XZmVW65ezR9d1lGIfIRZQwyQ6EUL0xZYSQy80AZiES"

# Create API client
client = tweepy.Client(bearer_token)

# df = pd.read_csv('data/notes-00000.tsv', sep='\t')
# print(df['tweetId'].head())
#
# tweetid = df['tweetId'][0]
# print(tweetid)
# tweetid = '1783159712986382830'
# 
# # Get tweet by ID
# tweet = client.get_tweet(tweetid, 
#                          tweet_fields=['author_id','created_at','public_metrics','source'],
#                          user_fields=['affiliation','public_metrics','username','verified','verified_type','parody'])

tweets_response = client.get_tweets(tweet_ids,
                                    tweet_fields=['author_id','created_at','public_metrics','source'],
                                    user_fields=['affiliation','public_metrics','username','verified','verified_type','parody'])


In [29]:
print(tweets_response)

Response(data=[<Tweet id=1730553246001902032 text='If you actually, no joke, like really believed in Santa as a kid you were kinda dumb https://t.co/WDXwR2VC2z'>, <Tweet id=1676130410399670272 text='Escape and Fight.'>, <Tweet id=1606013465126002690 text='Why is $TSLA plunging?\n\n@ElonMusk: "Full Self-Driving...is the difference between Tesla being worth a lot of money and being worth basically zero."\n\nMany competitors have driverless robotaxis but after 8 years of development FSD will still run down a child in a school crosswalk: https://t.co/fE2lQIwzIq'>, <Tweet id=1677475709202190344 text='On my watch, health care is a right not a privilege in this country.'>, <Tweet id=1712871805344506097 text='https://t.co/wPMtZqNeEa'>, <Tweet id=1705322338189705308 text="This weekend, New York will host Dave Portnoy's One Bite Pizza Festival.\n\nParticipating pizzerias have had to navigate buzz around the festival and backlash against the Barstool Sports founder's controversial remarks and all

In [30]:
# print(tweets_response.data)
data = list(tweets_response.data)
tweets_data = [dict(tweet_data) for tweet_data in data]
print(tweets_data)

[{'text': 'If you actually, no joke, like really believed in Santa as a kid you were kinda dumb https://t.co/WDXwR2VC2z', 'created_at': datetime.datetime(2023, 12, 1, 11, 43, 40, tzinfo=datetime.timezone.utc), 'public_metrics': {'retweet_count': 776, 'reply_count': 711, 'like_count': 8173, 'quote_count': 2654, 'bookmark_count': 945, 'impression_count': 24951874}, 'id': 1730553246001902032, 'edit_history_tweet_ids': [1730553246001902032], 'author_id': 1058348443410878464}, {'text': 'Escape and Fight.', 'created_at': datetime.datetime(2023, 7, 4, 7, 26, 45, tzinfo=datetime.timezone.utc), 'public_metrics': {'retweet_count': 1728, 'reply_count': 1044, 'like_count': 23500, 'quote_count': 644, 'bookmark_count': 473, 'impression_count': 157297791}, 'id': 1676130410399670272, 'edit_history_tweet_ids': [1676130410399670272], 'author_id': 1100982077636280321}, {'text': 'Why is $TSLA plunging?\n\n@ElonMusk: "Full Self-Driving...is the difference between Tesla being worth a lot of money and being 

In [31]:
def flatten_tweet_data(data):
    flat_data = {}
    for key, value in data.items():
        if isinstance(value, dict):
            flat_data.update(flatten_tweet_data(value))
        else:
            flat_data[key] = value
    return flat_data

In [32]:
flat_data = [flatten_tweet_data(tweet_data) for tweet_data in tweets_data]
# convert datetime to string
for tweet_data in flat_data:
    tweet_data['created_at'] = tweet_data['created_at'].strftime('%Y-%m-%d %H:%M:%S')
print(len(flat_data))
# print(json.dumps(flat_data, indent=2))

89


In [24]:
# save to json
with open('first_tweets.json', 'w') as f:
    json.dump(flat_data, f, indent=2)

In [34]:
# load data into database
# 
# 
# # --- Database connection details ---
DB_HOST = 'localhost'
DB_USER = 'gmaayan'
DB_PASSWORD = 'j.UyNY.3NsP6vrdfy3LNq.2f'
DB_NAME = 'tweets'

# --- Connect to the database ---
db_connection = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    charset="utf8mb4",
    collation="utf8mb4_unicode_ci"
)

# Create a cursor object
cursor = db_connection.cursor()

insert_query = """
INSERT INTO tweets (
    created_at,
    retweet_count,
    reply_count,
    like_count,
    quote_count,
    bookmark_count,
    impression_count,
    text,
    edit_history_tweet_ids,
    author_id,
    id
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each tweet into the database
for tweet in flat_data:
    edit_history_json = json.dumps(tweet["edit_history_tweet_ids"])
    values = (
        tweet['created_at'],
        tweet['retweet_count'],
        tweet['reply_count'],
        tweet['like_count'],
        tweet['quote_count'],
        tweet['bookmark_count'],
        tweet['impression_count'],
        tweet['text'],
        edit_history_json,
        tweet['author_id'],
        tweet['id']
    )

    cursor.execute(insert_query, values)

# Commit the transaction
db_connection.commit()